[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter_garch_seminar_notebook.ipynb)

---

# Seminar: GARCH Models - Practice Exercises

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Exercises Overview

1. **Exercise 1:** ARCH-LM Testing and GARCH Estimation
2. **Exercise 2:** Model Comparison (GARCH vs GJR-GARCH vs EGARCH)
3. **Exercise 3:** Real Data Analysis - S&P 500
4. **Exercise 4:** Volatility Forecasting and VaR
5. **Exercise 5:** Bitcoin Volatility Analysis

## Setup

In [ ]:
# Install packages if needed
try:
    from arch import arch_model
except ImportError:
    !pip install arch yfinance --quiet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from arch import arch_model
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf

try:
    import yfinance as yf
    HAS_YF = True
except:
    HAS_YF = False

# Plotting style
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['savefig.transparent'] = True
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22'}

print("Setup complete!")

---

## Exercise 1: ARCH-LM Testing and GARCH Estimation

**Objective:** Learn to detect ARCH effects and estimate basic GARCH models.

**Tasks:**
1. Simulate a GARCH(1,1) process
2. Apply the ARCH-LM test
3. Estimate the model and compare with true parameters

In [ ]:
# Task 1: Simulate GARCH(1,1)
np.random.seed(123)
n = 2000

# True parameters
omega_true = 0.00005
alpha_true = 0.12
beta_true = 0.83

# Initialize
sigma2 = np.zeros(n)
epsilon = np.zeros(n)
sigma2[0] = omega_true / (1 - alpha_true - beta_true)

# Generate process
for t in range(1, n):
    sigma2[t] = omega_true + alpha_true * epsilon[t-1]**2 + beta_true * sigma2[t-1]
    epsilon[t] = np.sqrt(sigma2[t]) * np.random.randn()

returns = epsilon * 100

print("True GARCH(1,1) Parameters:")
print(f"  ω = {omega_true}")
print(f"  α = {alpha_true}")
print(f"  β = {beta_true}")
print(f"  α + β = {alpha_true + beta_true}")

In [ ]:
# Task 2: Apply ARCH-LM test
print("ARCH-LM Test Results:")
print("="*50)

for lags in [5, 10, 20]:
    lm_stat, lm_pval, _, _ = het_arch(returns - returns.mean(), nlags=lags)
    result = "ARCH present" if lm_pval < 0.05 else "No ARCH"
    print(f"  Lags={lags:2d}: LM={lm_stat:8.2f}, p-value={lm_pval:.6f} → {result}")

# Visual check
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(returns, color=COLORS['blue'], linewidth=0.5)
axes[0].set_title('Returns (Volatility Clustering Visible)', fontweight='bold')

plot_acf(returns**2, lags=30, ax=axes[1], color=COLORS['red'],
         vlines_kwargs={'color': COLORS['red']})
axes[1].set_title('ACF of Squared Returns', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Task 3: Estimate GARCH(1,1) and compare
model = arch_model(returns, vol='Garch', p=1, q=1, dist='normal')
results = model.fit(disp='off')

print("\nParameter Comparison:")
print("="*50)
print(f"{'Parameter':<12} {'True':>10} {'Estimated':>12} {'Std.Err':>10}")
print("-"*50)
print(f"{'omega':<12} {omega_true:>10.6f} {results.params['omega']:>12.6f} {results.std_err['omega']:>10.6f}")
print(f"{'alpha':<12} {alpha_true:>10.4f} {results.params['alpha[1]']:>12.4f} {results.std_err['alpha[1]']:>10.4f}")
print(f"{'beta':<12} {beta_true:>10.4f} {results.params['beta[1]']:>12.4f} {results.std_err['beta[1]']:>10.4f}")

print("\n✓ Estimates are close to true values!")

---

## Exercise 2: Model Comparison

**Objective:** Compare symmetric and asymmetric GARCH models.

**Tasks:**
1. Simulate data with leverage effect
2. Estimate GARCH, GJR-GARCH, and EGARCH
3. Compare using AIC/BIC

In [ ]:
# Task 1: Simulate GJR-GARCH with leverage
np.random.seed(789)
n = 2000

omega = 0.00005
alpha = 0.04
gamma = 0.10  # Leverage parameter
beta = 0.88

sigma2_lev = np.zeros(n)
eps_lev = np.zeros(n)
sigma2_lev[0] = omega / (1 - alpha - gamma/2 - beta)

for t in range(1, n):
    I = 1 if eps_lev[t-1] < 0 else 0
    sigma2_lev[t] = omega + alpha * eps_lev[t-1]**2 + gamma * eps_lev[t-1]**2 * I + beta * sigma2_lev[t-1]
    eps_lev[t] = np.sqrt(sigma2_lev[t]) * np.random.randn()

returns_lev = eps_lev * 100

print("True GJR-GARCH Parameters:")
print(f"  α = {alpha}, γ = {gamma}, β = {beta}")
print(f"  Positive shock impact: {alpha}")
print(f"  Negative shock impact: {alpha + gamma}")

In [ ]:
# Task 2: Estimate all three models
print("Model Estimation and Comparison")
print("="*60)

# GARCH(1,1)
model_garch = arch_model(returns_lev, vol='Garch', p=1, q=1, dist='t')
res_garch = model_garch.fit(disp='off')

# GJR-GARCH
model_gjr = arch_model(returns_lev, vol='Garch', p=1, o=1, q=1, dist='t')
res_gjr = model_gjr.fit(disp='off')

# EGARCH
model_egarch = arch_model(returns_lev, vol='EGARCH', p=1, q=1, dist='t')
res_egarch = model_egarch.fit(disp='off')

# Task 3: Compare
print(f"\n{'Model':<15} {'AIC':>10} {'BIC':>10} {'LogLik':>12}")
print("-"*50)
print(f"{'GARCH(1,1)':<15} {res_garch.aic:>10.2f} {res_garch.bic:>10.2f} {res_garch.loglikelihood:>12.2f}")
print(f"{'GJR-GARCH':<15} {res_gjr.aic:>10.2f} {res_gjr.bic:>10.2f} {res_gjr.loglikelihood:>12.2f}")
print(f"{'EGARCH':<15} {res_egarch.aic:>10.2f} {res_egarch.bic:>10.2f} {res_egarch.loglikelihood:>12.2f}")

# Identify best
models = {'GARCH': res_garch.aic, 'GJR-GARCH': res_gjr.aic, 'EGARCH': res_egarch.aic}
best = min(models, key=models.get)
print(f"\n→ Best model by AIC: {best}")

In [ ]:
# Check if leverage is significant in GJR-GARCH
print("\nGJR-GARCH: Leverage Effect Test")
print("="*50)

gamma_est = res_gjr.params['gamma[1]']
gamma_se = res_gjr.std_err['gamma[1]']
gamma_tstat = gamma_est / gamma_se
gamma_pval = 2 * (1 - stats.norm.cdf(abs(gamma_tstat)))

print(f"  γ estimate: {gamma_est:.4f}")
print(f"  Std error:  {gamma_se:.4f}")
print(f"  t-stat:     {gamma_tstat:.2f}")
print(f"  p-value:    {gamma_pval:.4f}")
print(f"\n  Leverage significant? {'Yes ✓' if gamma_pval < 0.05 else 'No'}")

---

## Exercise 3: Real Data Analysis - S&P 500

**Objective:** Apply GARCH modeling to real financial data.

**Tasks:**
1. Download S&P 500 data
2. Calculate returns and test for ARCH effects
3. Estimate and diagnose GARCH models

In [ ]:
# Task 1: Get S&P 500 data
if HAS_YF:
    try:
        sp500 = yf.download('^GSPC', start='2015-01-01', end='2024-12-31', progress=False)
        returns_sp = 100 * sp500['Adj Close'].pct_change().dropna()
        DATA_SOURCE = "Yahoo Finance"
    except:
        HAS_YF = False

if not HAS_YF:
    # Simulated realistic S&P 500 data
    np.random.seed(2020)
    n = 2500
    omega, alpha, gamma, beta = 0.01, 0.08, 0.10, 0.88
    sigma2 = np.zeros(n)
    eps = np.zeros(n)
    sigma2[0] = omega / (1 - alpha - gamma/2 - beta)
    for t in range(1, n):
        I = 1 if eps[t-1] < 0 else 0
        sigma2[t] = omega + alpha * eps[t-1]**2 + gamma * eps[t-1]**2 * I + beta * sigma2[t-1]
        eps[t] = np.sqrt(sigma2[t]) * np.random.standard_t(df=6)
    returns_sp = pd.Series(eps, index=pd.date_range('2015-01-01', periods=n, freq='B'))
    DATA_SOURCE = "Simulated"

print(f"Data source: {DATA_SOURCE}")
print(f"Sample size: {len(returns_sp)} observations")
print(f"Period: {returns_sp.index[0].date()} to {returns_sp.index[-1].date()}")
print(returns_sp.describe())

In [ ]:
# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(returns_sp.index, returns_sp, color=COLORS['blue'], linewidth=0.5)
axes[0].axhline(y=0, color='black', linewidth=0.5)
axes[0].set_ylabel('Return (%)')
axes[0].set_title(f'S&P 500 Daily Returns ({DATA_SOURCE})', fontweight='bold')

# Rolling 20-day volatility
rolling_vol = returns_sp.rolling(window=20).std()
axes[1].plot(rolling_vol.index, rolling_vol, color=COLORS['red'], linewidth=0.8)
axes[1].set_ylabel('Rolling Volatility (%)')
axes[1].set_xlabel('Date')
axes[1].set_title('20-Day Rolling Volatility', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Task 2: Test for ARCH effects
print("\nARCH-LM Test on S&P 500 Returns")
print("="*50)

for lags in [5, 10, 20]:
    lm_stat, lm_pval, _, _ = het_arch(returns_sp - returns_sp.mean(), nlags=lags)
    print(f"  Lags={lags:2d}: LM={lm_stat:8.2f}, p-value={lm_pval:.6f}")

print("\n→ Strong ARCH effects detected!")

In [ ]:
# Task 3: Estimate GJR-GARCH with Student-t
model_sp = arch_model(returns_sp, vol='Garch', p=1, o=1, q=1, dist='t')
res_sp = model_sp.fit(disp='off')

print(res_sp.summary())

In [ ]:
# Interpretation
print("\nParameter Interpretation")
print("="*50)

alpha_sp = res_sp.params['alpha[1]']
gamma_sp = res_sp.params['gamma[1]']
beta_sp = res_sp.params['beta[1]']
nu_sp = res_sp.params['nu']

persistence = alpha_sp + gamma_sp/2 + beta_sp
half_life = np.log(0.5) / np.log(persistence) if persistence < 1 else float('inf')

print(f"  α = {alpha_sp:.4f} (news reaction)")
print(f"  γ = {gamma_sp:.4f} (leverage effect)")
print(f"  β = {beta_sp:.4f} (persistence)")
print(f"  ν = {nu_sp:.2f} (Student-t degrees of freedom)")
print(f"\n  Persistence (α + γ/2 + β) = {persistence:.4f}")
print(f"  Half-life = {half_life:.1f} days")
print(f"\n  Leverage ratio: {(alpha_sp + gamma_sp)/alpha_sp:.2f}x higher impact for negative shocks")

In [ ]:
# Diagnostics
std_resid_sp = res_sp.std_resid

print("\nDiagnostic Tests")
print("="*50)

# Ljung-Box on squared residuals
lb_test = acorr_ljungbox(std_resid_sp**2, lags=10, return_df=True)
print(f"Ljung-Box (z²): Q(10) = {lb_test['lb_stat'].iloc[-1]:.2f}, p-value = {lb_test['lb_pvalue'].iloc[-1]:.4f}")

# ARCH-LM on standardized residuals
lm_stat, lm_pval, _, _ = het_arch(std_resid_sp, nlags=5)
print(f"ARCH-LM (5 lags): LM = {lm_stat:.2f}, p-value = {lm_pval:.4f}")

if lm_pval > 0.05:
    print("\n✓ No remaining ARCH effects - model adequate!")
else:
    print("\n⚠️ Some ARCH effects remain - consider alternative specifications")

---

## Exercise 4: Volatility Forecasting and VaR

**Objective:** Generate volatility forecasts and calculate Value at Risk.

**Tasks:**
1. Forecast volatility 10 days ahead
2. Calculate 1-day and 10-day VaR
3. Backtest VaR

In [ ]:
# Task 1: Volatility forecast
horizon = 10
forecasts = res_sp.forecast(horizon=horizon)
vol_forecast = np.sqrt(forecasts.variance.values[-1, :])

print("Volatility Forecast (next 10 days)")
print("="*40)
for h in range(horizon):
    print(f"  Day {h+1}: {vol_forecast[h]:.3f}%")

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

# Last 50 days historical
hist_vol = res_sp.conditional_volatility
ax.plot(range(50), hist_vol[-50:], color=COLORS['blue'], label='Historical')
ax.plot(range(49, 49+horizon+1), np.concatenate([[hist_vol.iloc[-1]], vol_forecast]),
        color=COLORS['red'], linestyle='--', linewidth=2, label='Forecast')
ax.axvline(x=49, color='black', linestyle='-', alpha=0.3)
ax.set_xlabel('Days')
ax.set_ylabel('Volatility (%)')
ax.set_title('Volatility Forecast', fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2)

plt.tight_layout()
plt.show()

In [ ]:
# Task 2: Calculate VaR
portfolio_value = 1_000_000  # EUR
sigma_1day = vol_forecast[0] / 100

# Quantiles
nu = res_sp.params['nu']
q_95_norm = stats.norm.ppf(0.95)
q_99_norm = stats.norm.ppf(0.99)
q_95_t = stats.t.ppf(0.95, df=nu) * np.sqrt((nu-2)/nu)
q_99_t = stats.t.ppf(0.99, df=nu) * np.sqrt((nu-2)/nu)

print("Value at Risk Calculation")
print("="*50)
print(f"Portfolio: €{portfolio_value:,.0f}")
print(f"1-day volatility: {sigma_1day*100:.2f}%")

print("\n1-Day VaR:")
print(f"  Normal 95%: €{q_95_norm * sigma_1day * portfolio_value:,.0f}")
print(f"  Normal 99%: €{q_99_norm * sigma_1day * portfolio_value:,.0f}")
print(f"  Student-t 95%: €{q_95_t * sigma_1day * portfolio_value:,.0f}")
print(f"  Student-t 99%: €{q_99_t * sigma_1day * portfolio_value:,.0f}")

# 10-day VaR (sqrt(10) scaling)
print("\n10-Day VaR (scaled):")
print(f"  Normal 99%: €{q_99_norm * sigma_1day * portfolio_value * np.sqrt(10):,.0f}")
print(f"  Student-t 99%: €{q_99_t * sigma_1day * portfolio_value * np.sqrt(10):,.0f}")

In [ ]:
# Task 3: Simple VaR backtest
print("\nVaR Backtest (95% level)")
print("="*50)

# Calculate 1-day VaR for each day
cond_vol = res_sp.conditional_volatility / 100
VaR_95 = q_95_t * cond_vol

# Count violations (returns < -VaR)
returns_dec = returns_sp / 100
violations = returns_dec < -VaR_95
violation_rate = violations.mean()
expected_rate = 0.05

print(f"Expected violation rate: {expected_rate*100:.1f}%")
print(f"Actual violation rate: {violation_rate*100:.2f}%")
print(f"Number of violations: {violations.sum()} out of {len(violations)}")

# Kupiec test (proportion of failures)
n_total = len(violations)
n_viol = violations.sum()
LR = 2 * (n_viol * np.log(violation_rate/expected_rate) + 
          (n_total - n_viol) * np.log((1-violation_rate)/(1-expected_rate)))
pval = 1 - stats.chi2.cdf(LR, 1)

print(f"\nKupiec LR test: LR = {LR:.2f}, p-value = {pval:.4f}")
print(f"Conclusion: {'VaR model adequate ✓' if pval > 0.05 else 'VaR model inadequate ✗'}")

---

## Exercise 5: Bitcoin Volatility Analysis

**Objective:** Analyze cryptocurrency volatility and compare with traditional assets.

**Tasks:**
1. Download Bitcoin data and calculate returns
2. Estimate GARCH models
3. Compare volatility characteristics with S&P 500

In [ ]:
# Task 1: Get Bitcoin data
if HAS_YF:
    try:
        btc = yf.download('BTC-USD', start='2018-01-01', end='2024-12-31', progress=False)
        returns_btc = 100 * btc['Adj Close'].pct_change().dropna()
        BTC_SOURCE = "Yahoo Finance"
    except:
        HAS_YF = False

if not HAS_YF:
    # Simulate Bitcoin-like volatility (higher than S&P)
    np.random.seed(2021)
    n = 2000
    omega, alpha, gamma, beta = 0.5, 0.15, 0.08, 0.80
    sigma2 = np.zeros(n)
    eps = np.zeros(n)
    sigma2[0] = omega / (1 - alpha - gamma/2 - beta)
    for t in range(1, n):
        I = 1 if eps[t-1] < 0 else 0
        sigma2[t] = omega + alpha * eps[t-1]**2 + gamma * eps[t-1]**2 * I + beta * sigma2[t-1]
        eps[t] = np.sqrt(sigma2[t]) * np.random.standard_t(df=5)
    returns_btc = pd.Series(eps, index=pd.date_range('2018-01-01', periods=n, freq='B'))
    BTC_SOURCE = "Simulated"

print(f"Bitcoin data source: {BTC_SOURCE}")
print(f"Sample: {len(returns_btc)} observations")
print(f"\nBasic statistics:")
print(returns_btc.describe())

In [ ]:
# Visualize Bitcoin returns
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(returns_btc.index, returns_btc, color=COLORS['orange'], linewidth=0.5)
axes[0].axhline(y=0, color='black', linewidth=0.5)
axes[0].set_ylabel('Return (%)')
axes[0].set_title(f'Bitcoin Daily Returns ({BTC_SOURCE})', fontweight='bold')

axes[1].hist(returns_btc, bins=100, density=True, color=COLORS['orange'],
             alpha=0.7, edgecolor='white')
x = np.linspace(returns_btc.min(), returns_btc.max(), 100)
axes[1].plot(x, stats.norm.pdf(x, returns_btc.mean(), returns_btc.std()),
             color=COLORS['red'], linewidth=2)
axes[1].set_title(f'Distribution (Kurtosis = {stats.kurtosis(returns_btc)+3:.1f})', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Task 2: Estimate GARCH model for Bitcoin
model_btc = arch_model(returns_btc, vol='Garch', p=1, o=1, q=1, dist='t')
res_btc = model_btc.fit(disp='off')

print(res_btc.summary())

In [ ]:
# Task 3: Comparison with S&P 500
print("\nComparison: Bitcoin vs S&P 500")
print("="*60)

# Get comparable sample
common_start = max(returns_sp.index[0], returns_btc.index[0])
common_end = min(returns_sp.index[-1], returns_btc.index[-1])
sp_comp = returns_sp[(returns_sp.index >= common_start) & (returns_sp.index <= common_end)]
btc_comp = returns_btc[(returns_btc.index >= common_start) & (returns_btc.index <= common_end)]

print(f"\n{'Metric':<25} {'S&P 500':>12} {'Bitcoin':>12}")
print("-"*50)
print(f"{'Mean return (%)':<25} {sp_comp.mean():>12.3f} {btc_comp.mean():>12.3f}")
print(f"{'Volatility (%)':<25} {sp_comp.std():>12.3f} {btc_comp.std():>12.3f}")
print(f"{'Skewness':<25} {stats.skew(sp_comp):>12.3f} {stats.skew(btc_comp):>12.3f}")
print(f"{'Kurtosis':<25} {stats.kurtosis(sp_comp)+3:>12.3f} {stats.kurtosis(btc_comp)+3:>12.3f}")

# Model parameters
print(f"\n{'GARCH Parameters:':<25}")
print(f"{'α (news reaction)':<25} {res_sp.params['alpha[1]']:>12.4f} {res_btc.params['alpha[1]']:>12.4f}")
print(f"{'γ (leverage)':<25} {res_sp.params['gamma[1]']:>12.4f} {res_btc.params['gamma[1]']:>12.4f}")
print(f"{'β (persistence)':<25} {res_sp.params['beta[1]']:>12.4f} {res_btc.params['beta[1]']:>12.4f}")
print(f"{'ν (Student-t df)':<25} {res_sp.params['nu']:>12.2f} {res_btc.params['nu']:>12.2f}")

# Calculate persistence
pers_sp = res_sp.params['alpha[1]'] + res_sp.params['gamma[1]']/2 + res_sp.params['beta[1]']
pers_btc = res_btc.params['alpha[1]'] + res_btc.params['gamma[1]']/2 + res_btc.params['beta[1]']
print(f"{'Persistence (α+γ/2+β)':<25} {pers_sp:>12.4f} {pers_btc:>12.4f}")

print("\nKey findings:")
print(f"  - Bitcoin volatility is ~{btc_comp.std()/sp_comp.std():.1f}x higher than S&P 500")
print(f"  - Bitcoin has {'higher' if res_btc.params['alpha[1]'] > res_sp.params['alpha[1]'] else 'lower'} α (faster reaction to news)")
print(f"  - Bitcoin has {'weaker' if res_btc.params['gamma[1]'] < res_sp.params['gamma[1]'] else 'stronger'} leverage effect")

---

## Summary

### What We Learned

1. **ARCH-LM test** detects heteroskedasticity in financial returns

2. **GARCH(1,1)** is sufficient for most applications
   - α measures news reaction
   - β measures persistence
   - α + β < 1 for stationarity

3. **Asymmetric models** (GJR, EGARCH) capture leverage effect
   - γ > 0 in GJR-GARCH indicates leverage
   - γ < 0 in EGARCH indicates leverage

4. **Student-t distribution** better captures fat tails

5. **VaR calculation** uses volatility forecasts
   - Student-t gives higher VaR than normal

6. **Cryptocurrencies** have higher volatility but similar GARCH dynamics